# Analyse van AskDelphi Relations & Tags endpoint
Dit notebook ondersteunt het verkennen, analyseren en valideren van de Relations & Tags functionaliteit binnen AskDelphi.  
De module biedt helpermethoden voor het toevoegen, beheren en inspecteren van relaties en tags die aan AskDelphi‑topics gekoppeld zijn.  
Het doel van dit notebook is om:

- inzicht te krijgen in de datastructuren en API‑interacties rondom relations & tags
- voorbeelddata te analyseren en patronen te ontdekken
- helpermethoden te testen en gedrag te evalueren
- experimentele of onderzoeksgerichte analyses uit te voeren ter verbetering van de contentstructuur in AskDelphi

Dit notebook vormt daarmee een flexibel startpunt voor verdere exploratie of debugging van de Relations & Tags endpoint.

### Initialisatie omgeving

In [1]:
import sys
import uuid
import json
from pathlib import Path

project_root = Path.cwd().parent          # ../ vanaf Notebooks/
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [2]:
from pathlib import Path

# Vind de projectroot vanuit Notebooks/
project_root = Path.cwd().parent
src_path = project_root / "src"

# Voeg src/ toe aan modulezoekpad
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

print("Project root:", project_root)
print("SRC path:", src_path)
print("SRC path is in sys.path?", str(src_path) in sys.path)

Project root: /Users/baasa03/projects/ask-delphi-api
SRC path: /Users/baasa03/projects/ask-delphi-api/src
SRC path is in sys.path? True


### Connectie met AskDelphi opzetten

In [3]:
from askdelphi.authentication import AskDelphiClient
client = AskDelphiClient()
client.authenticate()   # pakt automatisch portal code uit .env

AskDelphi Client loaded.
Parsed tenant/project/acl from CMS URL
Loaded cached tokens.
AUTHENTICATION STARTED
Trying cached tokens...


/Users/baasa03/projects/ask-delphi-api/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Editing API token status: 200
Editing API token acquired.
SUCCESS using cached tokens!


True

### Tag mapping

In [13]:
# "tags": [
#       {
#         "tagId": "00000000-0000-0000-0000-000000000000",
#         "enforcedByAcl": false,
#         "isTopicListFilter": false,
#         "hierarchyTopicId": "81edcdae-337a-4c11-a275-8f2a85be1033",
#         "hierarchyTopicTitle": "Ketens",
#         "hierarchyNodeId": "88595ffc-4343-4343-9f13-8871665c6445",
#         "hierarchyNodeTitle": "Inning en betalingsverkeer",
#         "pathToNode": "",
#         "hierarchyParentNodeId": "88595ffc-4343-4343-9f13-8871665c6445"
#       }

KNOWN_HIERARCHY = {    
    "Ketens": {        
        "Inning en betalingsverkeer": {           
            "hierarchyTopicId": "81edcdae-337a-4c11-a275-8f2a85be1033",
            "hierarchyNodeId": "88595ffc-4343-4343-9f13-8871665c6445"        
        }   
    }
}

def build_hierarchy_tag_from_known(title: str, node: str):
    meta = KNOWN_HIERARCHY[title][node]
    return {
        "tagId": "00000000-0000-0000-0000-000000000000",
        "enforcedByAcl": False,
        "isTopicListFilter": False,
        "hierarchyTopicId": meta["hierarchyTopicId"],
        "hierarchyNodeId": meta["hierarchyNodeId"],
        "hierarchyTopicTitle": "Ketens",
        "hierarchyNodeTitle": "Inning en betalingsverkeer",
        "pathToNode": "",
        "hierarchyParentNodeId": meta["hierarchyNodeId"]
    }

In [5]:
from askdelphi.project import Project
from askdelphi.topictools import TopicTools
from askdelphi.relation import Relation
from askdelphi.workflow import Workflow
workflow = Workflow(client)
project = Project(client)
topic = TopicTools(client, project)
relation = Relation(client)

### Creatie Digicoach

In [6]:
# Constant
DIGICOACH_NAME = "Digicoach test arie 6feb"
TASK_NAME = "Taak test 6feb"
PREDEFINED_SEARCH_NAME = "Voorgedefinieerde zoekopdracht 6feb"

# Create Voorgedefinieerde zoekopdracht topic
topic_id_predefined_search = topic.topic_upload(PREDEFINED_SEARCH_NAME, "Pre-defined search")
topic_version_id_predefined_search = topic.get_topicVersionId(topic_id_predefined_search)
print(f"Created Voorgedefinieerde zoekopdracht topic : {topic_id_predefined_search}")

# Create Digicoach topic
topic_id_digicoach = str(uuid.uuid4())      
topicTitle = DIGICOACH_NAME      
topicTypeId = project.get_topic_type_id("Digitale Coach Procespagina")     
parentTopicId = topic_id_predefined_search
parentTopicRelationTypeId = relation.get_relation_type_id(topic_id_predefined_search, topic_version_id_predefined_search,"Voorgedefinieerde zoekopdracht")
parentTopicVersionId = topic_version_id_predefined_search
relation.add_topic_with_relation(client, topic_id_digicoach, topicTitle, topicTypeId, parentTopicId, parentTopicRelationTypeId, parentTopicVersionId)
print(f"Created Digicoach topic : {topic_id_digicoach}")

# Get Digicoach topic version ID
topic_version_id_digicoach = topic.get_topicVersionId(topic_id_digicoach)

# Tag Digitale Coach Procespagina
data = build_hierarchy_tag_from_known("Ketens", "Inning en betalingsverkeer")
endpoint = f"/v2/tenant/{{tenantId}}/project/{{projectId}}/acl/{{aclEntryId}}/topic/{topic_id_digicoach}/topicVersion/{topic_version_id_digicoach}/tag"
client._request("POST", endpoint, json_data=data)

# Create Task topic
topic_id_task = str(uuid.uuid4())
topicTitle = TASK_NAME       
topicTypeId = project.get_topic_type_id("Task")       
parentTopicId = topic_id_digicoach
parentTopicRelationTypeId = relation.get_relation_type_id(topic_id_digicoach, topic_version_id_digicoach, "Taak")
parentTopicVersionId = topic_version_id_digicoach
relation.add_topic_with_relation(client, topic_id_task, topicTitle, topicTypeId, parentTopicId, parentTopicRelationTypeId, parentTopicVersionId)
print(f"Created Task topic : {topic_id_task}")

# Checkin Voorgedefinieerde zoekopdracht topic
topic.checkin(topic_id_predefined_search)
print(f"Checked in Voorgedefinieerde zoekopdracht topic : {topic_id_predefined_search}")

# Checkin Digicoach topic
topic.checkin(topic_id_digicoach)
print(f"Checked in Digicoach topic : {topic_id_digicoach}")

# # Delete Task topic
# topic_version_id_task = topic.get_topicVersionId(topic_id_task)
# topic.delete_topic(topic_id_task, topic_version_id_task)
# print(f"Deleted Task topic : {topic_id_task}")

# # Delete Digicoach topic
# topic.delete_topic(topic_id_digicoach, topic_version_id_digicoach)
# print(f"Deleted Digicoach topic : {topic_id_digicoach}")

# # Delete Voorgedefinieerde zoekopdracht topic
# topic.delete_topic(topic_id_predefined_search, topic_version_id_predefined_search)
# print(f"Deleted Voorgedefinieerde zoekopdracht topic : {topic_id_predefined_search}")


REQUEST: GET https://edit.api.askdelphi.com/v1/tenant/0be6d42b-c278-44e6-888e-ba122840d690/project/397296f6-20dd-45cd-8459-250db2725140/acl/4ecd88f2-979b-4fb0-a95d-175d499bc375/contentdesign
RESPONSE: 200 OK

REQUEST: POST https://edit.api.askdelphi.com/v4/tenant/0be6d42b-c278-44e6-888e-ba122840d690/project/397296f6-20dd-45cd-8459-250db2725140/acl/4ecd88f2-979b-4fb0-a95d-175d499bc375/topic
JSON body: {
  "topicTitle": "Voorgedefinieerde zoekopdracht 6feb",
  "topicTypeId": "72aaefbb-1c09-4f89-8830-54194b03523b"
}
RESPONSE: 200 OK

REQUEST: POST https://edit.api.askdelphi.com/v1/tenant/0be6d42b-c278-44e6-888e-ba122840d690/project/397296f6-20dd-45cd-8459-250db2725140/acl/4ecd88f2-979b-4fb0-a95d-175d499bc375/topic/43f67e19-f350-4622-8466-46834e9d52ac/workflowstate
JSON body: {
  "action": 1
}
RESPONSE: 200 OK
Created Voorgedefinieerde zoekopdracht topic : 43f67e19-f350-4622-8466-46834e9d52ac

REQUEST: GET https://edit.api.askdelphi.com/v1/tenant/0be6d42b-c278-44e6-888e-ba122840d690/proje

### Publiceer Digicoach

In [7]:
#  Creates a workflow transition request for predefined_search topic.
request_id = workflow.create_workflow_transition_request(topic_id_predefined_search)
transitions_model = workflow.get_workflow_transition_request_transitions_model(request_id)
workflow.update_workflow_transition_request(request_id, transitions_model)
workflow.approve_workflow_transition_request(request_id)

#  Creates a workflow transition request for digicoach topic.
request_id = workflow.create_workflow_transition_request(topic_id_digicoach)
transitions_model = workflow.get_workflow_transition_request_transitions_model(request_id)
workflow.update_workflow_transition_request(request_id, transitions_model)
workflow.approve_workflow_transition_request(request_id)

#  Creates a workflow transition request for task topic.
request_id = workflow.create_workflow_transition_request(topic_id_task)
transitions_model = workflow.get_workflow_transition_request_transitions_model(request_id)
workflow.update_workflow_transition_request(request_id, transitions_model)
workflow.approve_workflow_transition_request(request_id)


REQUEST: POST https://edit.api.askdelphi.com/v1/tenant/0be6d42b-c278-44e6-888e-ba122840d690/project/397296f6-20dd-45cd-8459-250db2725140/acl/4ecd88f2-979b-4fb0-a95d-175d499bc375/workflow-transition-request/topic/43f67e19-f350-4622-8466-46834e9d52ac
JSON body: {
  "Url": "https://digitalecoach.askdelphi.com/cms/tenant/0be6d42b-c278-44e6-888e-ba122840d690/project/397296f6-20dd-45cd-8459-250db2725140/acl/4ecd88f2-979b-4fb0-a95d-175d499bc375"
}
RESPONSE: 200 OK

REQUEST: GET https://edit.api.askdelphi.com/v1/tenant/0be6d42b-c278-44e6-888e-ba122840d690/project/397296f6-20dd-45cd-8459-250db2725140/acl/4ecd88f2-979b-4fb0-a95d-175d499bc375/workflow-transition-request/9926ee32-fc8a-4f7a-9c71-ba57b10fa1b1/transitions
RESPONSE: 200 OK

REQUEST: PUT https://edit.api.askdelphi.com/v1/tenant/0be6d42b-c278-44e6-888e-ba122840d690/project/397296f6-20dd-45cd-8459-250db2725140/acl/4ecd88f2-979b-4fb0-a95d-175d499bc375/workflow-transition-request/9926ee32-fc8a-4f7a-9c71-ba57b10fa1b1/transitions
JSON body:

{}

### Tag Digicoach

In [8]:
# topic_id_digicoach = "b656ff93-3437-4d3f-a638-dc2f9de645fb"

# # Checkout topic
# response = topic.checkout(topic_id_digicoach)
# topic_version_id_digicoach = topic.get_topicVersionId(topic_id_digicoach)
# print(f"Version id : {topic_version_id_digicoach}")

# # Tag Digitale Coach Procespagina -> Keten Inning en betalingsverkeer
# data = {
#    "tagId": "00000000-0000-0000-0000-000000000000",
#     "enforcedByAcl": False,
#     "isTopicListFilter": False,
#     "hierarchyTopicId": "81edcdae-337a-4c11-a275-8f2a85be1033",
#     "hierarchyTopicTitle": "Ketens",
#     "hierarchyNodeId": "88595ffc-4343-4343-9f13-8871665c6445",
#     "hierarchyNodeTitle": "Inning en betalingsverkeer",
#     "pathToNode": "",
#     "hierarchyParentNodeId": "88595ffc-4343-4343-9f13-8871665c6445"
# }

# endpoint = f"/v2/tenant/{{tenantId}}/project/{{projectId}}/acl/{{aclEntryId}}/topic/{topic_id_digicoach}/topicVersion/{topic_version_id_digicoach}/tag"
# client._request("POST", endpoint, json_data=data)

# # Checkin topic
# topic.checkin(topic_id_digicoach)
# topic_version_id_digicoach = topic.get_topicVersionId(topic_id_digicoach)
# print(f"Version id after checkin : {topic_version_id_digicoach}")

In [14]:
# Checkout Digicoach topic
print(f"Version id Digicoach Before checkout : {topic_version_id_digicoach}")

topic_id_digicoach_new = topic.get_topicVersionId(topic_id_digicoach)
print(f"Checked out Digicoach topic : {topic_id_digicoach_new}")

# Tag Digitale Coach Procespagina
data = build_hierarchy_tag_from_known("Ketens", "Inning en betalingsverkeer")
endpoint = f"/v2/tenant/{{tenantId}}/project/{{projectId}}/acl/{{aclEntryId}}/topic/{topic_id_digicoach}/topicVersion/{topic_id_digicoach_new}/tag"
client._request("POST", endpoint, json_data=data)

# Checkin Digicoach topic
response = topic.checkin(topic_id_digicoach)
print(response)

Version id Digicoach Before checkout : 10adfeb4-8273-4b58-add9-25283c426cf6

REQUEST: POST https://edit.api.askdelphi.com/v1/tenant/0be6d42b-c278-44e6-888e-ba122840d690/project/397296f6-20dd-45cd-8459-250db2725140/acl/4ecd88f2-979b-4fb0-a95d-175d499bc375/topic/1de66f6f-f3cb-40db-b2d0-b879f474f892/workflowstate
JSON body: {
  "action": 1
}
RESPONSE: 200 OK
Checked out Digicoach topic : 86fca465-db8b-4154-9b74-393ec432b060

REQUEST: POST https://edit.api.askdelphi.com/v2/tenant/0be6d42b-c278-44e6-888e-ba122840d690/project/397296f6-20dd-45cd-8459-250db2725140/acl/4ecd88f2-979b-4fb0-a95d-175d499bc375/topic/1de66f6f-f3cb-40db-b2d0-b879f474f892/topicVersion/86fca465-db8b-4154-9b74-393ec432b060/tag
JSON body: {
  "tagId": "00000000-0000-0000-0000-000000000000",
  "enforcedByAcl": false,
  "isTopicListFilter": false,
  "hierarchyTopicId": "81edcdae-337a-4c11-a275-8f2a85be1033",
  "hierarchyNodeId": "88595ffc-4343-4343-9f13-8871665c6445",
  "hierarchyTopicTitle": "Ketens",
  "hierarchyNodeTitle

In [16]:
import json
# Type Digitale Coach Procespagina
endpoint = f"/v1/tenant/{{tenantId}}/project/{{projectId}}/acl/{{aclEntryId}}/topic/{topic_id_digicoach}/tag"

result = client._request("GET", endpoint)
print(json.dumps(result, indent=2, ensure_ascii=False))


REQUEST: GET https://edit.api.askdelphi.com/v1/tenant/0be6d42b-c278-44e6-888e-ba122840d690/project/397296f6-20dd-45cd-8459-250db2725140/acl/4ecd88f2-979b-4fb0-a95d-175d499bc375/topic/1de66f6f-f3cb-40db-b2d0-b879f474f892/tag
RESPONSE: 200 OK
{
  "success": true,
  "errorCode": null,
  "errorMessage": null,
  "response": {
    "tags": [
      {
        "tagId": "00000000-0000-0000-0000-000000000000",
        "enforcedByAcl": false,
        "isTopicListFilter": false,
        "hierarchyTopicId": "81edcdae-337a-4c11-a275-8f2a85be1033",
        "hierarchyTopicTitle": "Ketens",
        "hierarchyNodeId": "88595ffc-4343-4343-9f13-8871665c6445",
        "hierarchyNodeTitle": "Inning en betalingsverkeer",
        "pathToNode": "",
        "hierarchyParentNodeId": "88595ffc-4343-4343-9f13-8871665c6445"
      }
    ]
  }
}


In [ ]:
# Topics
# endpoint = f"/v1/tenant/{{tenantId}}/project/{{projectId}}/acl/{{aclEntryId}}/topiclist"

# all_topics = []
# page = 0
# page_size = 1000

# while True:
#     payload = {
#         "page": page,
#         "pageSize": page_size
#     }

#     resp = client._request("POST", endpoint, json_data=payload)
#     batch = resp["response"]["topicList"]["result"]

#     # print(json.dumps(batch, indent=2, ensure_ascii=False))

#     if not batch:
#         break

#     all_topics.extend(batch)
#     page += 1

# print(len(all_topics))  # should be 626



REQUEST: POST https://edit.api.askdelphi.com/v1/tenant/0be6d42b-c278-44e6-888e-ba122840d690/project/397296f6-20dd-45cd-8459-250db2725140/acl/4ecd88f2-979b-4fb0-a95d-175d499bc375/topiclist
JSON body: {
  "page": 0,
  "pageSize": 1000
}
RESPONSE: 200 OK

REQUEST: POST https://edit.api.askdelphi.com/v1/tenant/0be6d42b-c278-44e6-888e-ba122840d690/project/397296f6-20dd-45cd-8459-250db2725140/acl/4ecd88f2-979b-4fb0-a95d-175d499bc375/topiclist
JSON body: {
  "page": 1,
  "pageSize": 1000
}
RESPONSE: 200 OK

REQUEST: POST https://edit.api.askdelphi.com/v1/tenant/0be6d42b-c278-44e6-888e-ba122840d690/project/397296f6-20dd-45cd-8459-250db2725140/acl/4ecd88f2-979b-4fb0-a95d-175d499bc375/topiclist
JSON body: {
  "page": 2,
  "pageSize": 1000
}
RESPONSE: 200 OK
1252
